# EddyPro API

This notebook gives an outline for working with the API.

* First step is to define your input primary parameters.  You need to give a siteID, everything else is *optional*.

    * kwargs (dict of all optional settings):\
        * sourceDir: if not provided, assumes data are already in Outputs folder
        * dateRange (if not provided, will run all files in the current year)
        * fileType (defaults to .ghg) must specify otherwise for 
        * metadataTemplate (only required for .dat files)



In [1]:
import eddyProAPI
import importlib
importlib.reload(eddyProAPI)
kwargs = {
    'siteID':'BB',
    'dateRange':['2024-04-1 10:00','2024-04-3 04:00'],
    # 'sourceDir':"C:/highfreq/BB/raw/2024/04",
    'reset':True,
    'metaDataUpdates':'C:/highfreq/BB/Manual_Metadata_Updates.csv',
    'priority':'High Priority',
    'runMode':'1',
    # 'processes':1,
    'biometUser':True
}
kwargs['userDefinedEddyProSettings'] = {
        # 'Project':{
        #     'hf_meth':'4'
        #     }
            }
eP = eddyProAPI.eddyProAPI(**kwargs)

Deleting contents of : Y://BB/metadata/
Deleting contents of : Y://BB/eddyProAPIOutputs/
Biomet user: Querying database for up-to-date biomet data
Generating requested files tasks for BB over: ['2024-04-01 10:00' '2024-04-03 04:00']


ImportError: Missing optional dependency 'fsspec'.  Use pip or conda to install fsspec.

In [8]:
import pandas as pd
import os
df = pd.DataFrame(data={'a':[1,2],'b':[3,4]})
# pth = 'Y://BB//metadata//BB_biometData_202405010000_202405040000.csv'
pth = 'Y://BB/metadata/test.csv'
pth = os.path.abspath(pth)
# df.to_csv(pth,index=False)
print(pth)

Y:\BB\metadata\test.csv


## Example Command Line Call


'One winmet call this before activating .venv
```
Set-ExecutionPolicy Unrestricted -Scope Process
```


```
.\.venv\Scripts\activate                                                               
```                                                                                                     
```
py eddyProAPI.py --siteID BB --biometUser True --sourceDir Y:/BB/raw/2024/ --metaDataUpdates Y:/BB/Manual_Metadata_Updates.csv --runMode full --eddyProStaticConfig ini_files/ClosedPathStandard.eddypro
```

```
py eddyProAPI.py --siteID BBS --biometUser True --sourceDir C:/data-dump/BBS/EC_Station/2024/ --metaDataTemplate C:/highfreq/BBS/TOA5_BBS.FLUX_2023_06_14_1500.metadata --runMode 1 --eddyProStaticConfig ini_files/OpenPathStandard.eddypro --fileType dat --timeShift 30
```

# siteID = 'BBS'
# kwargs = {
#     'dateRange':['2023-06-01','2024-05-31'],
#     'sourceDir':"X:/BBS/EC_Station/2024/20240516/",
#     'searchTag':"BBS.FLUX",
#     'timeShift':30,
#     'fileType':'dat',
#     'metaDataTemplate':'Y:/BBS/TOA5_BBS.FLUX_2023_06_14_1500.metadata',
# }


In [8]:
import pandas as pd
pth = "C:\\temp\\BB\\metadata\\rawDataStatistics.csv"
kwargs = {
    'parse_dates': [0],
    'index_col': [0],
    'header': [0,1,2]
}
df = pd.read_csv(pth,**kwargs)
df.columns[0]

eP.metaDataValues.columns

MultiIndex([(   'Site',           'site_name'),
            (   'Site',            'altitude'),
            (   'Site',            'latitude'),
            (   'Site',           'longitude'),
            (   'Site',       'canopy_height'),
            (   'Site', 'displacement_height'),
            (   'Site',    'roughness_length'),
            ('Station',        'station_name'),
            ('Station',           'logger_id'),
            ('Station',   'logger_sw_version'),
            ...
            ( 'Custom',  'col_49_header_name'),
            ( 'Custom',  'col_50_header_name'),
            ( 'Custom',  'col_51_header_name'),
            ( 'Custom',  'col_52_header_name'),
            ( 'Custom',  'col_53_header_name'),
            ( 'Custom',  'col_54_header_name'),
            ( 'Custom',  'col_55_header_name'),
            ( 'Custom',  'col_56_header_name'),
            ( 'Timing',     'expectedSamples'),
            (  'group',                  'ID')],
           length=837)

In [22]:
Temp = eP.rawDataStatistics.copy()
cols = Temp.columns
nuCols = []
for c in cols:
    c = [a if a != '' else eP.config['stringTags']['NaN'] for a in c]
    nuCols.append(tuple(c))
# # cols[cols == ''] = eP.config['stringTags']['NaN']
# # cols
# Temp.columns.set_levels(cols,level=1)
Temp.columns = nuCols
Temp.columns

Index([           ('Seconds', '~', 'count'),
                    ('Seconds', '~', 'max'),
                   ('Seconds', '~', 'mean'),
                 ('Seconds', '~', 'median'),
                    ('Seconds', '~', 'min'),
                    ('Seconds', '~', 'std'),
              ('Nanoseconds', '~', 'count'),
                ('Nanoseconds', '~', 'max'),
               ('Nanoseconds', '~', 'mean'),
             ('Nanoseconds', '~', 'median'),
       ...
       ('CH4 Drop Rate (%)', '~', 'median'),
          ('CH4 Drop Rate (%)', '~', 'min'),
          ('CH4 Drop Rate (%)', '~', 'std'),
                      ('CHK', '~', 'count'),
                        ('CHK', '~', 'max'),
                       ('CHK', '~', 'mean'),
                     ('CHK', '~', 'median'),
                        ('CHK', '~', 'min'),
                        ('CHK', '~', 'std'),
                       ('group', 'ID', '~')],
      dtype='object', length=325)

In [1]:

# Example run for BB on one day in 2024
siteID = 'BB'
kwargs = {
    'dateRange':['2024-04-01','2024-04-02'],
    'sourceDir':"Y:/BB/raw/2024/",
    # 'reset':True,
    'metaDataUpdates':'Y:/BB/Manual_Metadata_Updates.csv',
    'priority':'High Priority'
}

# # # Example run for All BBS data from the Datadump Folder 20240516
# siteID = 'BBS'
# kwargs = {
#     'dateRange':['2023-06-01','2024-05-31'],
#     'sourceDir':"X:/BBS/EC_Station/2024/20240516/",
#     'searchTag':"BBS.FLUX",
#     'timeShift':30,
#     'fileType':'dat',
#     'metaDataTemplate':'Y:/BBS/TOA5_BBS.FLUX_2023_06_14_1500.metadata',
# }


# # Cospectral correction options
# hf_meth = {
#     'Moncrieff et al. (1997)':'1',
#     'Horst (1997)':'2',
#     'Ibrom et al. (2007)':'3',
#     'Fratini et al. (2012)':'4', # Recommended for closed path systems, when sufficient data (>1 month) are available
#     'Massman (2000, 2001)':'5'
# }

# if siteID == 'BBS':
#     userDefinedEddyProSettings = {
#         'Project':{
#             'hf_meth':'1'
#             },
#         'RawProcess_Settings':{
#             'v_offset':0.08,
#             'u_offset':-0.04
#             },
#         }
#     kwargs['userDefinedEddyProSettings'] = userDefinedEddyProSettings

# Check the configurations

In [6]:
# import eddyProAPI
# import importlib
# importlib.reload(eddyProAPI)
# # kwargs['siteID'] = siteID
# kwargs = {
#     'siteID':'BB',
#     'dateRange':['2024-04-01','2024-04-15'],
#     'sourceDir':"Y:/BB/raw/2024/04",
#     'reset':True,
#     'metaDataUpdates':'Y:/BB/Manual_Metadata_Updates.csv',
#     'priority':'High Priority'
# }
# kwargs['userDefinedEddyProSettings'] = {
#         'Project':{
#             'hf_meth':'4'
#             }}

# eP = eddyProAPI.eddyProAPI(**kwargs)
# eP.config['BiometUser']

Deleting contents of : C:/Users/labuser/Temp//BB/metadata/
Deleting contents of : C:/Users/labuser/Temp//BB/raw/


{'Biomet.Net': 'C:/Users/labuser/Biomet.net/', 'Database': 'W:/'}

## Run PreProcessing

In [1]:
import eddyProAPI
import importlib
importlib.reload(eddyProAPI)
kwargs = {
    'siteID':'BB',
    'dateRange':['2024-04-15','2024-04-17'],
    'sourceDir':"C:/highfreq/BB/raw/2024/04",
    # 'reset':True,
    'metaDataUpdates':'C:/highfreq/BB/Manual_Metadata_Updates.csv',
    'priority':'High Priority',
    'biometUser':True
}
kwargs['userDefinedEddyProSettings'] = {
        'Project':{
            'hf_meth':'4'
            }}
pre = eddyProAPI.preProcessing(**kwargs)

Initializing tasks for BB over: ['2024-04-15 00:00' '2024-04-17 00:00']
See output: C:/temp//BB/metadata//BB_biometData_202404150000_202404170000.csv
canopy_height missing, outputting NaNs
See output: C:/temp//BB/metadata//BB_dynamicMetadata_202404150000_202404170000.csv
All tasks completed successfully
fileInventory {'parse_dates': [0], 'index_col': [0], 'filepath_or_buffer': 'C:/temp//BB/metadata/fileInventory.csv'}
rawDataStatistics {'parse_dates': [0], 'index_col': [0], 'header': [0, 1, 2], 'filepath_or_buffer': 'C:/temp//BB/metadata/rawDataStatistics.csv'}
metaDataValues {'parse_dates': [0], 'index_col': [0], 'header': [0, 1], 'filepath_or_buffer': 'C:/temp//BB/metadata/metaDataValues.csv'}
configurationGroups {'index_col': [0], 'header': [0, 1, 2], 'filepath_or_buffer': 'C:/temp//BB/metadata/configurationGroups.csv'}
Searching C:/highfreq/BB/raw/2024/04
[████████████████████████████████████████████████████████████] 110/110

Files Search Complete, time elapsed:  0.914
Reading Data

# Run EddyPro in Parallel over date range

In [4]:

import eddyProAPI
import importlib
importlib.reload(eddyProAPI)

kwargs['dumpToBiometDatabase']=True
# if kwargs['reset'] == True:
#     kwargs['reset'] = False
eP = eddyProAPI.runEP(**kwargs)

Initializing tasks for BB over: ['2024-04-15 00:00' '2024-04-17 00:00']
See output: C:/temp//BB/metadata//BB_biometData_202404150000_202404170000.csv
canopy_height missing, outputting NaNs
See output: C:/temp//BB/metadata//BB_dynamicMetadata_202404150000_202404170000.csv
All tasks completed successfully
Warning, available data in group 1 is below recommended size for selected settings.
1 group_1_rp_A group_1_yyyy-mm-ddTHHMM??_LI-7200.ghg 2024-04-15 00:00:00 2024-04-15 11:30:00 23
Creating c:\MM_Py\EddyPro_API\temp\group_1_rp_A.eddypro for 23 files
1 group_1_fcc group_1_yyyy-mm-ddTHHMM??_LI-7200.ghg 2024-04-15 00:00:00 2024-04-15 11:30:00 23
Creating c:\MM_Py\EddyPro_API\temp\group_1_fcc.eddypro for 23 files
Initiating EddyPro Runs for group on 1 cores at High Priority priority


In [20]:
import batchProcessing
import importlib
importlib.reload(batchProcessing)
import os

d_out = os.path.abspath('Y:\\BB\\eddyProAPIOutputs')
d_in = os.path.abspath('C:\\Users\\labuser\\EddyPro_API\\temp ')
print(d_in,d_out)
print(os.path.isdir(d_in))
print(os.path.isdir(d_out))
batchProcessing.pasteWithSubprocess(d_in,d_out,option='xcopy',Verbose=True)

C:\Users\labuser\EddyPro_API\temp Y:\BB\eddyProAPIOutputs
True
True
CompletedProcess(args=['move', 'C:\\Users\\labuser\\EddyPro_API\\temp', 'Y:\\BB\\eddyProAPIOutputs'], returncode=1, stdout=b'', stderr=b'Access is denied.\r\n')
